In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

Read the data

In [2]:
df = pd.read_csv('final_data.csv')
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Weekly_Sales_MA4,Weekly_Sales_Std2,Weekly_Sales_Std4,Weekly_Sales_Lag52,Weekly_Sales_Seasonal,Type_Encoded,Size_Normalized,store_cont,dept_cont,Set
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2,0.239209,0.012609,0.022371,train
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,...,NaN,14930.552614,NaN,NaN,NaN,2,0.239209,0.023291,0.041323,train
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,...,NaN,3142.340109,NaN,NaN,NaN,2,0.239209,0.021043,0.037334,train
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,...,32990.77,15692.120759,12832.106391,NaN,NaN,2,0.239209,0.009816,0.017416,train
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,...,32216.62,1714.281396,13554.047185,NaN,NaN,2,0.239209,0.011042,0.019592,train


In [3]:
train  = df[df['Set'] == 'train']
test  = df[df['Set'] == 'test']
print(train.shape)
print(test.shape)


(374203, 54)
(47367, 54)


Filtering for store and dept combinations with full time period

In [4]:
# Find all (Store, Dept) combinations with full 143 weeks of data
full_combinations = (
    df.groupby(['Store', 'Dept'])
    .size()
    .reset_index(name='Count')
    .query('Count == 143')[['Store', 'Dept']]
)

# Filter df to only those (Store, Dept) combinations
df = df.merge(full_combinations, on=['Store', 'Dept'], how='inner')
print(df.shape)
print(df[['Store', 'Dept']].value_counts())

(380380, 54)
Store  Dept
1      1       143
28     83      143
       87      143
       90      143
       91      143
              ... 
14     90      143
       91      143
       92      143
       93      143
45     97      143
Name: count, Length: 2660, dtype: int64


In [5]:
df.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'Has_Markdown_Info',
       'Year', 'Month', 'WeekOfYear', 'DayOfWeek', 'DayOfMonth', 'Quarter',
       'IsWeekend', 'IsSuperBowl', 'IsLaborDay', 'IsThanksgiving',
       'IsChristmas', 'PreHolidayWeek', 'PostHolidayWeek', 'IsBlackFriday',
       'BlackFridayWeek', 'ChristmasWeek', 'NewYearWeek', 'IsSummer',
       'IsWinter', 'IsSpring', 'IsFall', 'IsEndOfMonth', 'IsBeginningOfMonth',
       'Weekly_Sales_Lag1', 'Weekly_Sales_Lag2', 'Weekly_Sales_Lag4',
       'Weekly_Sales_MA2', 'Weekly_Sales_MA4', 'Weekly_Sales_Std2',
       'Weekly_Sales_Std4', 'Weekly_Sales_Lag52', 'Weekly_Sales_Seasonal',
       'Type_Encoded', 'Size_Normalized', 'store_cont', 'dept_cont', 'Set'],
      dtype='object')

**econ** - Temperature, Fuel_Price, CPI, Unemployment

**ops** - MarkDown1, MarkDown2, MarkDown3, MarkDown4, MarkDown5, Type, Size, 
        Has_Markdown_Info, Type_Encoded, Size_Normalized, store_cont, dept_cont

**cal**  - IsHoliday, Year, Month, WeekOfYear, DayOfWeek, DayOfMonth, Quarter,
       IsWeekend, IsSuperBowl, IsLaborDay, IsThanksgiving,
       IsChristmas, PreHolidayWeek, PostHolidayWeek, IsBlackFriday,
       BlackFridayWeek, ChristmasWeek, NewYearWeek, IsSummer,
       IsWinter, IsSpring, IsFall, IsEndOfMonth, IsBeginningOfMonth,
       Weekly_Sales_Lag1, Weekly_Sales_Lag2, Weekly_Sales_Lag4,
       Weekly_Sales_MA2, Weekly_Sales_MA4, Weekly_Sales_Std2, Weekly_Sales_Std4, Weekly_Sales_Lag52, Weekly_Sales_Seasonal

exp1_df = base_variables (store, dept, date, weekly_sales) + econ  
exp2_df = base_variables (store, dept, date, weekly_sales) + ops  
exp3_df = base_variables (store, dept, date, weekly_sales) + cal  
exp4_df = base_variables (store, dept, date, weekly_sales) + econ + ops  
exp5_df = base_variables (store, dept, date, weekly_sales) + econ + cal  
exp6_df = base_variables (store, dept, date, weekly_sales) + ops + cal  
exp7_df = base_variables (store, dept, date, weekly_sales) + econ + ops + cal  

Creating data for Experiment matrix

In [6]:
# Define the base and feature group variables
base_variables = ['Store', 'Dept', 'Date', 'Weekly_Sales', 'Set']

econ_vars = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

ops_vars = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 
            'Type', 'Size', 'Has_Markdown_Info', 'Type_Encoded', 'Size_Normalized', 
            'store_cont', 'dept_cont']

cal_vars = ['IsHoliday', 'Year', 'Month', 'WeekOfYear', 'DayOfWeek', 'DayOfMonth', 'Quarter',
            'IsWeekend', 'IsSuperBowl', 'IsLaborDay', 'IsThanksgiving',
            'IsChristmas', 'PreHolidayWeek', 'PostHolidayWeek', 'IsBlackFriday',
            'BlackFridayWeek', 'ChristmasWeek', 'NewYearWeek', 'IsSummer',
            'IsWinter', 'IsSpring', 'IsFall', 'IsEndOfMonth', 'IsBeginningOfMonth',
            'Weekly_Sales_Lag1', 'Weekly_Sales_Lag2', 'Weekly_Sales_Lag4',
            'Weekly_Sales_MA2', 'Weekly_Sales_MA4', 'Weekly_Sales_Std2', 
            'Weekly_Sales_Std4', 'Weekly_Sales_Lag52', 'Weekly_Sales_Seasonal']

# --- Create the 7 experimental DataFrames ---
exp1_df = df[base_variables + econ_vars]
exp2_df = df[base_variables + ops_vars]
exp3_df = df[base_variables + cal_vars]
exp4_df = df[base_variables + econ_vars + ops_vars]
exp5_df = df[base_variables + econ_vars + cal_vars]
exp6_df = df[base_variables + ops_vars + cal_vars]
exp7_df = df[base_variables + econ_vars + ops_vars + cal_vars]

# --- Verification ---
print("--- DataFrame Creation Verification ---")
dataframes = {
    "Experiment 1 (Base + Econ)": exp1_df,
    "Experiment 2 (Base + Ops)": exp2_df,
    "Experiment 3 (Base + Cal)": exp3_df,
    "Experiment 4 (Base + Econ + Ops)": exp4_df,
    "Experiment 5 (Base + Econ + Cal)": exp5_df,
    "Experiment 6 (Base + Ops + Cal)": exp6_df,
    "Experiment 7 (Base + Econ + Ops + Cal)": exp7_df
}

for name, exp_df in dataframes.items():
    print(f"\n{name}:")
    print(f"  Shape: {exp_df.shape}")
    # print(f"  Columns: {list(exp_df.columns)}") # Uncomment to see all columns



--- DataFrame Creation Verification ---

Experiment 1 (Base + Econ):
  Shape: (380380, 9)

Experiment 2 (Base + Ops):
  Shape: (380380, 17)

Experiment 3 (Base + Cal):
  Shape: (380380, 38)

Experiment 4 (Base + Econ + Ops):
  Shape: (380380, 21)

Experiment 5 (Base + Econ + Cal):
  Shape: (380380, 42)

Experiment 6 (Base + Ops + Cal):
  Shape: (380380, 50)

Experiment 7 (Base + Econ + Ops + Cal):
  Shape: (380380, 54)


Build XGBoost model to forecast weekly sales

In [9]:
# !pip install xgboost

In [14]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np
import pandas as pd

def train_evaluate_xgboost(df, experiment_name):
    """
    Trains an XGBoost model, evaluates it at the Store-Dept level, 
    and returns a DataFrame of the metrics.
    """
    print(f"--- Running {experiment_name} ---")

    # --- 1. Data Preparation ---
    cols_to_drop = ['Date', 'Set', 'Weekly_Sales', 'Type']
    cols_to_drop_existing = [col for col in cols_to_drop if col in df.columns]
    
    X = df.drop(columns=cols_to_drop_existing)
    y = df['Weekly_Sales']
    
    X = X.fillna(0)

    train_indices = df['Set'] == 'train'
    test_indices = df['Set'] == 'test'
    
    X_train, y_train = X[train_indices], y[train_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    
    # --- 2. Model Training ---
    xgbr = xgb.XGBRegressor(objective='reg:squarederror', 
                            n_estimators=100, 
                            learning_rate=0.1, 
                            max_depth=5, 
                            subsample=0.8, 
                            colsample_bytree=0.8,
                            random_state=42,
                            n_jobs=-1)

    print("Training XGBoost model...")
    xgbr.fit(X_train, y_train)

    # --- 3. Prediction ---
    print("Making predictions...")
    predictions = xgbr.predict(X_test)

    # --- 4. Evaluate at Store-Dept Level ---
    test_results_df = df.loc[test_indices, ['Store', 'Dept']].copy()
    test_results_df['Actual_Sales'] = y_test.values
    test_results_df['Predicted_Sales'] = predictions

    def calculate_metrics(group):
        rmse = np.sqrt(mean_squared_error(group['Actual_Sales'], group['Predicted_Sales']))
        mape = mean_absolute_percentage_error(group['Actual_Sales'], group['Predicted_Sales']) * 100
        return pd.Series({'RMSE': rmse, 'MAPE': mape})

    print("Calculating metrics at Store-Dept level...")
    store_dept_metrics = test_results_df.groupby(['Store', 'Dept']).apply(calculate_metrics).reset_index()
    
    print(f"Finished {experiment_name}.\n")
    
    return store_dept_metrics

# --- Loop through all experimental dataframes ---
all_results = []
for name, exp_df in dataframes.items():
    metrics_df = train_evaluate_xgboost(exp_df, name)
    if metrics_df is not None:
        metrics_df['Experiment'] = name
        all_results.append(metrics_df)

# --- Consolidate and Display Results ---
if all_results:
    final_results_df = pd.concat(all_results, ignore_index=True)

    print("\n--- Final Results Summary (Store-Dept Level) ---")
    print("Sample of detailed results:")
    print(final_results_df.head())

    print("\n--- Average Metrics per Experiment ---")
    average_metrics = final_results_df.groupby('Experiment')[['RMSE', 'MAPE']].mean().sort_values('RMSE')
    print(average_metrics)
else:
    print("No results were generated.")

--- Running Experiment 1 (Base + Econ) ---
Training XGBoost model...
Making predictions...
Calculating metrics at Store-Dept level...
Finished Experiment 1 (Base + Econ).

--- Running Experiment 2 (Base + Ops) ---
Training XGBoost model...
Making predictions...
Calculating metrics at Store-Dept level...
Finished Experiment 2 (Base + Ops).

--- Running Experiment 3 (Base + Cal) ---
Training XGBoost model...
Making predictions...
Calculating metrics at Store-Dept level...
Finished Experiment 3 (Base + Cal).

--- Running Experiment 4 (Base + Econ + Ops) ---
Training XGBoost model...
Making predictions...
Calculating metrics at Store-Dept level...
Finished Experiment 4 (Base + Econ + Ops).

--- Running Experiment 5 (Base + Econ + Cal) ---
Training XGBoost model...
Making predictions...
Calculating metrics at Store-Dept level...
Finished Experiment 5 (Base + Econ + Cal).

--- Running Experiment 6 (Base + Ops + Cal) ---
Training XGBoost model...
Making predictions...
Calculating metrics at S

In [ ]:
all_results

[      Store  Dept          RMSE        MAPE                  Experiment
 0         1     1   4623.636992   24.322697  Experiment 1 (Base + Econ)
 1         1     2   2718.085083    5.147581  Experiment 1 (Base + Econ)
 2         1     3  12225.022134   36.112129  Experiment 1 (Base + Econ)
 3         1     4   6563.550275   16.461551  Experiment 1 (Base + Econ)
 4         1     5   7714.536495   37.885837  Experiment 1 (Base + Econ)
 ...     ...   ...           ...         ...                         ...
 2655     45    91   8314.769279   51.479569  Experiment 1 (Base + Econ)
 2656     45    92  11087.061352   22.476247  Experiment 1 (Base + Econ)
 2657     45    93  13403.516065  448.309889  Experiment 1 (Base + Econ)
 2658     45    95   4654.679004    8.469080  Experiment 1 (Base + Econ)
 2659     45    97   2128.690688   31.997227  Experiment 1 (Base + Econ)
 
 [2660 rows x 5 columns],
       Store  Dept          RMSE       MAPE                 Experiment
 0         1     1  17639

In [23]:
final_results_df[(final_results_df['Store'] == 1)&(final_results_df['Dept'] == 4)]

,Store,Dept,RMSE,MAPE,Experiment
3,1,4,6563.550275,16.461551,Experiment 1 (Base + Econ)
2663,1,4,36018.078330,96.533036,Experiment 2 (Base + Ops)
5323,1,4,1010.983943,2.179304,Experiment 3 (Base + Cal)
7983,1,4,36198.626062,97.020737,Experiment 4 (Base + Econ + Ops)
10643,1,4,850.824177,1.774510,Experiment 5 (Base + Econ + Cal)
13303,1,4,5803.894517,15.266290,Experiment 6 (Base + Ops + Cal)
15963,1,4,5982.949829,15.572599,Experiment 7 (Base + Econ + Ops + Cal)
